In [ ]:
try:
    import google.colab
    !rm -rf /content/SportWarzSim
    !git clone https://github.com/daltontf/SportWarzSim.git 
    %cd SportWarzSim
except ImportError:
    pass

In [ ]:
from sportwarz import load_counties_geojson

counties_geojson = load_counties_geojson()

In [ ]:
from sportwarz import load_counties_data

co_data_frame = load_counties_data(counties_geojson)

### Blank Sandbox

In [ ]:
from sportwarz import load_leagues, Team, Coordinates

league_name = "MLB"

leagues = {
    league_name: { 
            "json": {
            "league_name": "Major League Baseball", 
            "weight": 0.7,
            "teams": [ ]
        } 
    }     
}

In [ ]:
from ipyleaflet import Map, GeoJSON, Popup, FullScreenControl, basemaps

from sportwarz import render_map, reset_county_styles, heatmap_counties

reset_county_styles(counties_geojson)

map = render_map(leagues, league_name, counties_geojson, co_data_frame)

map

In [ ]:
from sportwarz import Team, Coordinates, LeagueJson,\
    calculate_distances, create_show_teams, compute_shares, add_team,\
    compute_output_dataframes, reset_county_styles, heatmap_counties

import json
import codecs
import ipywidgets as widgets

out = widgets.Output()
out.clear_output()

def add_new_team(button):
    button.disabled=True
    newTeam = Team(
        name = "Houston Aeros",
        L = 2.0,
        S = 3.0,
        venue = "Toyota Center",
        state = "Texas",
        color = "darkgreen",
        coordinates = Coordinates(
            lat = 29.750833,
            lon = -95.362222
    ))
    add_team(co_data_frame, leagues, league_name, [newTeam])

    reset_county_styles(counties_geojson)
    heatmap_counties(leagues, counties_geojson, league_name)
    map.remove_layer(map.layers[1])
    layer = GeoJSON(data = counties_geojson, 
        hover_style = {"fillColor": "white"}
    )    
    layer.on_click(create_show_teams(map, leagues, co_data_frame))
    map.add(layer)
    button.disabled=False

name = widgets.Text()
venue = widgets.Text()
L = widgets.FloatText()
S = widgets.FloatText()
N = widgets.FloatText()
lat = widgets.FloatText()
lon = widgets.FloatText()
color = widgets.ColorPicker()
state = widgets.Text()
add_button = widgets.Button(description="Add Team")

add_button.on_click(add_new_team)

add_panel = widgets.GridBox([
    widgets.Label(value='Name:'), name,
    widgets.Label(value='Venue:'), venue,
    widgets.Label(value='L:'), L,
    widgets.Label(value='S:'), S,
    widgets.Label(value='N:'), N,
    widgets.Label(value='Lat:'), lat,
    widgets.Label(value='Lon:'), lon,
    widgets.Label(value='Color:'), color,
    widgets.Label(value='State:'), state,
    add_button, widgets.Label(),
],layout=widgets.Layout(grid_template_columns="repeat(2, 100px)"))

upload = widgets.FileUpload(
    accept='.json', 
    multiple= False  
)

@out.capture()
def on_upload_change(change):
    upload.disabled = True
    file_info = change["new"][0]
    content = codecs.decode(file_info.content, encoding="utf-8")
    league_json: LeagueJson = json.loads(content)
    league_name = league_json["league_name"]
    if not league_name in leagues:
        leagues[league_name] = {}
    leagues[league_name]["json"] = league_json
    calculate_distances(leagues, co_data_frame) 
    compute_shares(leagues, co_data_frame)  
    compute_output_dataframes(leagues, co_data_frame) 
    
    reset_county_styles(counties_geojson)
    heatmap_counties(leagues, counties_geojson, league_name)
    map.remove_layer(map.layers[1])
    layer = GeoJSON(data = counties_geojson, 
        hover_style = {"fillColor": "white"}
    )    
    layer.on_click(create_show_teams(map, leagues, co_data_frame))
    map.add(layer)
    upload.disabled=False   

upload.observe(on_upload_change, names=["value"])

upload_panel = widgets.GridBox([
    widgets.Label(value='File Upload:'), upload,    
], layout=widgets.Layout(grid_template_columns="repeat(2, 100px)"))

tab = widgets.Tab()
tab.children = [add_panel, upload_panel]
tab.titles =   ["Add", "Upload"]
tab

In [ ]:
out